In [1]:
%cd ..


E:\PROJECTS\Video Membership App\Video-Membership-Webapp


In [2]:
from app import db
from app.users.models import User
from app.videos.models import Video
from app.playlists.models import Playlist
from cassandra.cqlengine.management import sync_table

In [3]:
from app.indexing.client import get_index, update_index
from app.indexing.schemas import *

In [4]:
db.get_session()
sync_table(Playlist)
sync_table(User)
sync_table(Video)

In [5]:
import uuid
import json
from pydantic import BaseModel, Field, validator, root_validator
from typing import Optional

In [6]:
video_obj = dict(Video.objects.first())
video_obj

{'host_id': 'OQ1CwPhE8KQ',
 'db_id': UUID('7c67024e-8358-11ec-a71b-088fc304d5ee'),
 'host_service': 'youtube',
 'title': ' Call of Duty: Vanguard , Reveal Trailer',
 'url': 'https://youtu.be/OQ1CwPhE8KQ',
 'user_id': UUID('4f8e6bde-7056-11ec-90e6-90ccdf07a52a')}

In [7]:
class VideoIndexSchema(BaseModel):
    objectID : str = Field(alias="host_id")
    objectType: str = "Videos"
    title : Optional[str]
    path : str = Field(alias="host_id")
        
    @validator("path")
    def set_path(cls, v, values, **kwargs):
        host_id = v
        return f"/videos/{host_id}"

In [8]:
VideoIndexSchema(**video_obj).dict()

{'objectID': 'OQ1CwPhE8KQ',
 'objectType': 'Videos',
 'title': ' Call of Duty: Vanguard , Reveal Trailer',
 'path': '/videos/OQ1CwPhE8KQ'}

In [9]:
playlist_obj = dict(Playlist.objects.first())
playlist_obj

{'db_id': UUID('6be2ba23-7ba1-11ec-b5c0-088fc304d5ee'),
 'user_id': UUID('4f8e6bde-7056-11ec-90e6-90ccdf07a52a'),
 'updated': datetime.datetime(2022, 2, 17, 15, 22, 50, 737000),
 'host_ids': ['Axmg1E4HrVE', 'Axmg1E4HrVE'],
 'title': 'New Album 1'}

In [10]:
class PlaylistIndexSchema(BaseModel):
    objectID : uuid.UUID = Field(alias="db_id")
    objectType: str = "Videos"
    title : Optional[str]
    path : str = Field(default = "/")
    
    @root_validator
    def set_defaults(cls, values):
        objectID = values.get("objectID")
        values['objectID'] = str(objectID)
        values['path'] = f"/playlists/{objectID}"
        return values

In [11]:
PlaylistIndexSchema(**playlist_obj).dict()

{'objectID': '6be2ba23-7ba1-11ec-b5c0-088fc304d5ee',
 'objectType': 'Videos',
 'title': 'New Album 1',
 'path': '/playlists/6be2ba23-7ba1-11ec-b5c0-088fc304d5ee'}

In [12]:
video_q = [dict(x) for x in Video.objects.all()]
video_dataset = [VideoIndexSchema(**x).dict() for x in video_q]
video_dataset

[{'objectID': 'OQ1CwPhE8KQ',
  'objectType': 'Videos',
  'title': ' Call of Duty: Vanguard , Reveal Trailer',
  'path': '/videos/OQ1CwPhE8KQ'},
 {'objectID': 'cK4iAjzAoas',
  'objectType': 'Videos',
  'title': "Assassin's Creed Origins Cinematic Trailer",
  'path': '/videos/cK4iAjzAoas'},
 {'objectID': 'eyDBCowbKJs',
  'objectType': 'Videos',
  'title': 'Call of Duty: Vanguard - Angry Review',
  'path': '/videos/eyDBCowbKJs'},
 {'objectID': '4cJpiOPKH14',
  'objectType': 'Videos',
  'title': 'Star Wars Eclipse – Official Cinematic Reveal Trailer',
  'path': '/videos/4cJpiOPKH14'},
 {'objectID': '8D9pPtOgIFQ',
  'objectType': 'Videos',
  'title': 'Achilles: Legends Untold - Official Story Trailer',
  'path': '/videos/8D9pPtOgIFQ'},
 {'objectID': 'nOMIwsupy9k',
  'objectType': 'Videos',
  'title': 'Black Myth: Wukong - Official Unreal Engine 5 Gameplay Trailer',
  'path': '/videos/nOMIwsupy9k'},
 {'objectID': 'UxDWGW7Z67I',
  'objectType': 'Videos',
  'title': 'Horizon Forbidden West - S

In [13]:
playlist_q = [dict(x) for x in Playlist.objects.all()]
playlist_dataset = [PlaylistIndexSchema(**x).dict() for x in playlist_q]
playlist_dataset

[{'objectID': '6be2ba23-7ba1-11ec-b5c0-088fc304d5ee',
  'objectType': 'Videos',
  'title': 'New Album 1',
  'path': '/playlists/6be2ba23-7ba1-11ec-b5c0-088fc304d5ee'},
 {'objectID': '5cc2a152-7b9f-11ec-9136-088fc304d5ee',
  'objectType': 'Videos',
  'title': 'New Album',
  'path': '/playlists/5cc2a152-7b9f-11ec-9136-088fc304d5ee'},
 {'objectID': '884180c1-7b78-11ec-92bd-088fc304d5ee',
  'objectType': 'Videos',
  'title': 'hello world',
  'path': '/playlists/884180c1-7b78-11ec-92bd-088fc304d5ee'},
 {'objectID': '73010141-7b9f-11ec-a63e-088fc304d5ee',
  'objectType': 'Videos',
  'title': 'New Album',
  'path': '/playlists/73010141-7b9f-11ec-a63e-088fc304d5ee'},
 {'objectID': '4b1a2bb2-7b78-11ec-b077-088fc304d5ee',
  'objectType': 'Videos',
  'title': 'hello world',
  'path': '/playlists/4b1a2bb2-7b78-11ec-b077-088fc304d5ee'},
 {'objectID': '7f47d8c3-7b76-11ec-a3ef-088fc304d5ee',
  'objectType': 'Videos',
  'title': 'hello world',
  'path': '/playlists/7f47d8c3-7b76-11ec-a3ef-088fc304d5ee

In [14]:
dataset = video_dataset + playlist_dataset
dataset

[{'objectID': 'OQ1CwPhE8KQ',
  'objectType': 'Videos',
  'title': ' Call of Duty: Vanguard , Reveal Trailer',
  'path': '/videos/OQ1CwPhE8KQ'},
 {'objectID': 'cK4iAjzAoas',
  'objectType': 'Videos',
  'title': "Assassin's Creed Origins Cinematic Trailer",
  'path': '/videos/cK4iAjzAoas'},
 {'objectID': 'eyDBCowbKJs',
  'objectType': 'Videos',
  'title': 'Call of Duty: Vanguard - Angry Review',
  'path': '/videos/eyDBCowbKJs'},
 {'objectID': '4cJpiOPKH14',
  'objectType': 'Videos',
  'title': 'Star Wars Eclipse – Official Cinematic Reveal Trailer',
  'path': '/videos/4cJpiOPKH14'},
 {'objectID': '8D9pPtOgIFQ',
  'objectType': 'Videos',
  'title': 'Achilles: Legends Untold - Official Story Trailer',
  'path': '/videos/8D9pPtOgIFQ'},
 {'objectID': 'nOMIwsupy9k',
  'objectType': 'Videos',
  'title': 'Black Myth: Wukong - Official Unreal Engine 5 Gameplay Trailer',
  'path': '/videos/nOMIwsupy9k'},
 {'objectID': 'UxDWGW7Z67I',
  'objectType': 'Videos',
  'title': 'Horizon Forbidden West - S

In [15]:
index = get_index()


In [16]:
#dir(index)

In [17]:
#idx_response = index.save_objects(dataset).wait()
#idx_response[0]

In [18]:
#len(idx_response[0]['objectIDs'])

In [19]:
updated_count = update_index()
updated_count

17

In [20]:
index.search("Call of Duty")

{'hits': [{'objectType': 'Videos',
   'title': 'Call of Duty: Vanguard - Angry Review',
   'path': '/videos/eyDBCowbKJs',
   'objectID': 'eyDBCowbKJs',
   '_highlightResult': {'objectType': {'value': 'Videos',
     'matchLevel': 'none',
     'matchedWords': []},
    'title': {'value': '<em>Call</em> <em>of</em> <em>Duty</em>: Vanguard - Angry Review',
     'matchLevel': 'full',
     'fullyHighlighted': False,
     'matchedWords': ['call', 'of', 'duty']},
    'path': {'value': '/videos/eyDBCowbKJs',
     'matchLevel': 'none',
     'matchedWords': []}}},
  {'objectType': 'Videos',
   'title': ' Call of Duty: Vanguard , Reveal Trailer',
   'path': '/videos/OQ1CwPhE8KQ',
   'objectID': 'OQ1CwPhE8KQ',
   '_highlightResult': {'objectType': {'value': 'Videos',
     'matchLevel': 'none',
     'matchedWords': []},
    'title': {'value': ' <em>Call</em> <em>of</em> <em>Duty</em>: Vanguard , Reveal Trailer',
     'matchLevel': 'full',
     'fullyHighlighted': False,
     'matchedWords': ['call', 